In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

ModuleNotFoundError: No module named 'sklearn'

/home/akhtyamovpavel/PythonEnvironments/ResearchEnv/bin/python


In [2]:
df = pd.read_csv('count_matrix_data/GSE96769_scATACseq_matrix.csv', index_col=0)

In [4]:
df2 = pd.read_csv('count_matrix_data/GSE96769_cell_names_matrix.csv')

In [5]:
df2

,cell,cell_id,cell_types
0,singles-BM0828-HSC-fresh-151027-1,cell_1,HSC
1,singles-BM0828-HSC-fresh-151027-2,cell_2,HSC
2,singles-BM0828-HSC-fresh-151027-3,cell_3,HSC
3,singles-BM0828-HSC-fresh-151027-4,cell_4,HSC
4,singles-BM0828-HSC-fresh-151027-5,cell_5,HSC
...,...,...,...
2948,singles-160822-BM1137-CMP-LS-92,cell_2949,CMP
2949,singles-160822-BM1137-CMP-LS-93,cell_2950,CMP
2950,singles-160822-BM1137-CMP-LS-94,cell_2951,CMP
2951,singles-160822-BM1137-CMP-LS-95,cell_2952,CMP


In [6]:
df.columns

Index(['cell_1', 'cell_2', 'cell_3', 'cell_4', 'cell_5', 'cell_6', 'cell_7',
       'cell_8', 'cell_9', 'cell_10',
       ...
       'cell_2944', 'cell_2945', 'cell_2946', 'cell_2947', 'cell_2948',
       'cell_2949', 'cell_2950', 'cell_2951', 'cell_2952', 'cell_2953'],
      dtype='object', length=2947)

In [7]:
transposed_df = df.T

In [8]:
transposed_df.columns

Index(['Peak_52', 'Peak_53', 'Peak_68', 'Peak_69', 'Peak_151', 'Peak_332',
       'Peak_351', 'Peak_459', 'Peak_461', 'Peak_556',
       ...
       'Peak_355879', 'Peak_373713', 'Peak_390422', 'Peak_407340',
       'Peak_407673', 'Peak_431281', 'Peak_472663', 'Peak_239251',
       'Peak_350151', 'Peak_383161'],
      dtype='object', length=489671)

In [9]:
transposed_df.index

Index(['cell_1', 'cell_2', 'cell_3', 'cell_4', 'cell_5', 'cell_6', 'cell_7',
       'cell_8', 'cell_9', 'cell_10',
       ...
       'cell_2944', 'cell_2945', 'cell_2946', 'cell_2947', 'cell_2948',
       'cell_2949', 'cell_2950', 'cell_2951', 'cell_2952', 'cell_2953'],
      dtype='object', length=2947)

In [10]:
indices = set(df2.cell_id) & set(transposed_df.index)

In [25]:
df2[df2.cell_id in indices]

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [11]:
df2_filtered = df2[df2.cell_id.isin(indices)]

In [12]:
types = df2_filtered.cell_types.astype('category')

In [13]:
y = types.cat.codes.ravel()

In [14]:
X = transposed_df.values

In [15]:
X.shape

(2947, 489671)

In [18]:
import gc

In [19]:
gc.collect()

144

In [22]:
transposed_df[transposed_df.columns[:10000]]

,Peak_52,Peak_53,Peak_68,Peak_69,Peak_151,Peak_332,Peak_351,Peak_459,Peak_461,Peak_556,...,Peak_180899,Peak_181138,Peak_181146,Peak_181185,Peak_181223,Peak_181251,Peak_181317,Peak_181358,Peak_181367,Peak_181409
cell_1,1,1,1,1,1,1,1,1,1,2,...,0,0,0,0,0,0,0,0,0,0
cell_2,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
cell_3,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,2,2,1,1
cell_4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,2,0,0,0
cell_5,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cell_2949,0,0,0,0,6,0,3,0,1,1,...,0,1,0,0,0,0,2,0,3,0
cell_2950,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cell_2951,0,0,0,1,5,0,2,0,0,2,...,0,1,0,0,0,0,0,0,3,0
cell_2952,0,0,0,0,2,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0


In [46]:
def preprocess(df, y, start_index, length):
    rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)

    # define Boruta feature selection method
    feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)

    # find all relevant features - 5 features should be selected
    feat_selector.fit(df.values[:, start_index:start_index + length], y)
    feat_columns = df.columns[start_index:start_index + length][feat_selector.support_]
    
    X_output = feat_selector.transform(df.values[:, start_index:start_index + length])
    
    return pd.DataFrame(X_output, columns=feat_columns)

In [33]:
import numpy as np

In [47]:
dfs = []

for i in range(0, 490000, 10000):
    df1 = preprocess(transposed_df, y, i, 10000)
    
    dfs.append(df1)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	10000
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	10000
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	10000
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	10000
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	10000
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	10000
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	10000
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	850
Rejected: 	9150
Iteration: 	9 / 100
Confirmed: 	254
Tentative: 	596
Rejected: 	9150
Iteration: 	10 / 100
Confirmed: 	254
Tentative: 	596
Rejected: 	9150
Iteration: 	11 / 100
Confirmed: 	254
Tentative: 	596
Rejected: 	9150
Iteration: 	12 / 100
Confirmed: 	277
Tentative: 	460
Rejected: 	9263
Iteration: 	13 / 100
Confirmed: 	277
Tentative: 	460
Rejected: 	9263
Iteration: 	14 / 100
Confirmed: 	277
Tentative: 	460
Rejected: 	9263
Iteration: 	15 / 100
Confirmed: 	277
Tentative: 	460
Rejected: 	92

In [48]:
dfs[0]

,Peak_10446,Peak_14143,Peak_20090,Peak_24815,Peak_32800,Peak_32906,Peak_40862,Peak_53349,Peak_57124,Peak_63555,...,Peak_165174,Peak_165848,Peak_167246,Peak_169740,Peak_171035,Peak_172086,Peak_172220,Peak_175767,Peak_177517,Peak_178376
0,1,1,1,1,1,1,1,1,4,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,1,1,1,2,1,3,1,1,1,1
3,0,0,0,1,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2942,0,2,1,2,2,2,3,1,1,0,...,1,0,0,1,3,1,2,0,1,0
2943,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2944,1,0,1,3,0,1,1,0,2,0,...,0,0,1,0,4,1,1,5,0,0
2945,0,0,1,1,0,0,1,0,0,0,...,1,1,0,0,3,0,0,0,0,0


In [49]:
transposed_df.index

Index(['cell_1', 'cell_2', 'cell_3', 'cell_4', 'cell_5', 'cell_6', 'cell_7',
       'cell_8', 'cell_9', 'cell_10',
       ...
       'cell_2944', 'cell_2945', 'cell_2946', 'cell_2947', 'cell_2948',
       'cell_2949', 'cell_2950', 'cell_2951', 'cell_2952', 'cell_2953'],
      dtype='object', length=2947)

In [50]:
dfs[0].index = transposed_df.index

In [51]:
dfs[0]

,Peak_10446,Peak_14143,Peak_20090,Peak_24815,Peak_32800,Peak_32906,Peak_40862,Peak_53349,Peak_57124,Peak_63555,...,Peak_165174,Peak_165848,Peak_167246,Peak_169740,Peak_171035,Peak_172086,Peak_172220,Peak_175767,Peak_177517,Peak_178376
cell_1,1,1,1,1,1,1,1,1,4,1,...,0,0,0,0,0,0,0,0,0,0
cell_2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cell_3,1,0,0,0,0,0,0,0,0,0,...,1,1,1,2,1,3,1,1,1,1
cell_4,0,0,0,1,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
cell_5,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cell_2949,0,2,1,2,2,2,3,1,1,0,...,1,0,0,1,3,1,2,0,1,0
cell_2950,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cell_2951,1,0,1,3,0,1,1,0,2,0,...,0,0,1,0,4,1,1,5,0,0
cell_2952,0,0,1,1,0,0,1,0,0,0,...,1,1,0,0,3,0,0,0,0,0


In [52]:
for i in range(len(dfs)):
    dfs[i].index = transposed_df.index

In [53]:
df_output = dfs[0]

In [54]:
df_output[dfs[1].columns] = dfs[1]

In [55]:
for i in range(len(dfs)):
    df_output[dfs[i].columns] = dfs[i]

In [56]:
df_output.shape

(2947, 9045)

In [59]:
df_output.T.to_csv('count_matrix_data/filtered.csv')

In [58]:
!head count_matrix_data/GSE96769_scATACseq_matrix.csv

"","cell_1","cell_2","cell_3","cell_4","cell_5","cell_6","cell_7","cell_8","cell_9","cell_10","cell_11","cell_12","cell_13","cell_14","cell_15","cell_16","cell_17","cell_18","cell_19","cell_20","cell_21","cell_22","cell_23","cell_24","cell_25","cell_26","cell_27","cell_28","cell_29","cell_30","cell_31","cell_32","cell_33","cell_34","cell_35","cell_36","cell_37","cell_38","cell_39","cell_40","cell_41","cell_42","cell_43","cell_44","cell_45","cell_46","cell_47","cell_48","cell_49","cell_50","cell_51","cell_52","cell_53","cell_54","cell_55","cell_56","cell_57","cell_58","cell_59","cell_60","cell_61","cell_62","cell_63","cell_64","cell_65","cell_66","cell_67","cell_68","cell_69","cell_70","cell_71","cell_72","cell_73","cell_74","cell_75","cell_76","cell_77","cell_78","cell_79","cell_80","cell_81","cell_82","cell_83","cell_84","cell_85","cell_86","cell_87","cell_88","cell_89","cell_90","cell_91","cell_92","cell_93","cell_94","cell_95","cell_96","cell_97","cell_98","cell_99","cell_100","cell

In [60]:
!head count_matrix_data/filtered.csv

,cell_1,cell_2,cell_3,cell_4,cell_5,cell_6,cell_7,cell_8,cell_9,cell_10,cell_11,cell_12,cell_13,cell_14,cell_15,cell_16,cell_17,cell_18,cell_19,cell_20,cell_21,cell_22,cell_23,cell_24,cell_25,cell_26,cell_27,cell_28,cell_29,cell_30,cell_31,cell_32,cell_33,cell_34,cell_35,cell_36,cell_37,cell_38,cell_39,cell_40,cell_41,cell_42,cell_43,cell_44,cell_45,cell_46,cell_47,cell_48,cell_49,cell_50,cell_51,cell_52,cell_53,cell_54,cell_55,cell_56,cell_57,cell_58,cell_59,cell_60,cell_61,cell_62,cell_63,cell_64,cell_65,cell_66,cell_67,cell_68,cell_69,cell_70,cell_71,cell_72,cell_73,cell_74,cell_75,cell_76,cell_77,cell_78,cell_79,cell_80,cell_81,cell_82,cell_83,cell_84,cell_85,cell_86,cell_87,cell_88,cell_89,cell_90,cell_91,cell_92,cell_93,cell_94,cell_95,cell_96,cell_97,cell_98,cell_99,cell_100,cell_101,cell_102,cell_103,cell_104,cell_105,cell_106,cell_107,cell_108,cell_109,cell_110,cell_111,cell_112,cell_113,cell_114,cell_115,cell_116,cell_117,cell_118,cell_119,cell_120,cell_121,cell_122,cell_123,

In [2]:
!wc count_matrix_data/filtered.csv

    9046     9046 53449076 count_matrix_data/filtered.csv


In [3]:
df_ = pd.read_csv('count_matrix_data/peaks_chrX.tsv', sep='\t')

NameError: name 'pd' is not defined

In [ ]:
 peak[df.index.isin(my_list)]